# NSFNets: 用于不可压缩 Navier-Stokes 方程求解的物理信息神经网络

## 环境安装

本案例要求 MindSpore >= 2.0.0 版本以调用如下接口: mindspore.jit, mindspore.jit_class, mindspore.data_sink。具体请查看MindSpore安装。

此外，你需要安装 MindFlow >=0.1.0 版本。如果当前环境还没有安装，请按照下列方式选择后端和版本进行安装。

In [ ]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## 背景介绍

在AI-CFD的工业应用方面，对NavierStokes方程的求解以及对多精度数据的融合的场景十分广泛，具有重要的经济和社会意义。求解不适定问题（例如部分边界条件缺失）或反演问题是其中的重点和难点之一，且往往代价高昂，需要推导适用于特定问题的不同公式、编写全新代码。如何用一套统一代码以相同计算代价解决上述问题亟需深入研究。在此，金晓威和李惠等使用物理信息神经网络（PINNs），通过自动微分将控制方程直接编码到深度神经网络中，以克服上述一些模拟不可压缩层流和湍流流动的限制。并开发了Navier-Stokes流动网络（NSFnets，Navier-Stokes flow nets）。

## 模型框架

模型框架图如下所示:

![NSFNet](images/NSFNet.png)

图中表示的是压力-速度形式的纳维-斯托克斯方程求解网络，利用神经网络的自动微分计算方程中所需要的偏导项，损失函数包括，边界条件损失，初始条件损失以及为了满足方程平衡的物理损失。

## 准备环节

实践前，确保已经正确安装合适版本的MindSpore。如果没有，可以通过：

* [MindSpore安装页面](https://www.mindspore.cn/install) 安装MindSpore。

## 数据集的准备

数据集可以通过提供的代码直接生成。

## 模型训练

引入代码包

In [2]:
import os
import argparse

import time
import numpy as np
import mindspore as ms
from mindspore import set_seed, context, nn, Tensor
from src.network import VPNSFNets
from src.datasets import read_training_data

模型相关参数的设置以及训练模型的定义

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument('--model_name', type=str, default='NSFNet')
parser.add_argument('--case', type=str, default='Three dimensional Beltrami flow')
parser.add_argument('--device', type=str, default='GPU')
parser.add_argument('--device_id', type=str, default='1')
parser.add_argument('--load_params', type=str, default=False)
parser.add_argument('--second_path', type=str, default='train')
parser.add_argument('--network_size', type=int, default=[4] + 10 * [100 * 1] + [4])
parser.add_argument('--learning_rate', type=int, default=[1.0e-03, 1.0e-04, 1.0e-05, 1.0e-06])
parser.add_argument('--epochs', type=int, default=[5e3, 5e3, 5e4, 5e4])
parser.add_argument('--batch_size', type=int, default=10000)
parser.add_argument('--re', type=int, default=1)
args = parser.parse_known_args()[0]
# args = parser.parse_args()

model_name = args.model_name
case = args.case
device = args.device
device_id = args.device_id
network_size = args.network_size
learning_rate = args.learning_rate
epochs = args.epochs
batch_size = args.batch_size
load_params = args.load_params
second_path = args.second_path
re = args.re

use_ascend = context.get_context(attr_key='device_target') == "Ascend"

if use_ascend:
    msfloat_type = ms.float16
    npfloat_type = np.float16
else:
    msfloat_type = ms.float32
    npfloat_type = np.float32

os.environ['CUDA_VISIBLE_DEVICES'] = device_id
context.set_context(mode=context.GRAPH_MODE, save_graphs=False, device_target=device)

x_b, y_b, z_b, t_b, x_i, y_i, z_i, t_i, u_b, v_b, w_b, u_i, v_i, w_i, x_f, y_f, z_f, t_f, X_min, X_max = read_training_data()

model = VPNSFNets(x_b, y_b, z_b, t_b, x_i, y_i, z_i, t_i, u_b, v_b, w_b, u_i, v_i, w_i, x_f, y_f, z_f, t_f, network_size, re, \
                  X_min, X_max, use_ascend, msfloat_type, npfloat_type, load_params, second_path)

设置种子

In [ ]:
np.random.seed(123456)
set_seed(123456)

代码训练与输出结果部分

In [ ]:
def train(model, NIter, lr, batch_size=batch_size):
    params = model.dnn.trainable_params()
    optimizer_Adam = nn.Adam(params, learning_rate=lr)
    grad_fn = ms.value_and_grad(model.loss_fn, None, optimizer_Adam.parameters, has_aux=True)
    model.dnn.set_train()
    N_data = model.t_f.shape[0]
    start_time = time.time()

    for epoch in range(1, 1+NIter):
        idx_data_0 = np.random.choice(N_data, batch_size)
        idx_data = Tensor(idx_data_0)
        x_batch = model.x_f[idx_data, :]
        y_batch = model.y_f[idx_data, :]
        z_batch = model.z_f[idx_data, :]
        t_batch = model.t_f[idx_data, :]
        (loss, loss_b, loss_i, loss_f), grads = grad_fn(model.xb, model.yb, model.zb, model.tb, model.xi, model.yi, model.zi, model.ti, x_batch, y_batch, z_batch, t_batch, model.ub, model.vb, model.wb, model.ui, model.vi, model.wi)
        optimizer_Adam(grads)

        if epoch % 10 == 0:
            elapsed = time.time() - start_time
            print('It: %d, Total_loss: %.3e, Loss_b: %.3e, Loss_i: %.3e, Loss_f: %.3e, Time: %.2f' %\
                    (epoch, loss.item(), loss_b.item(), loss_i.item(), loss_f.item(), elapsed))
            loss_history_adam_pretrain = np.empty([0])
            loss_b_history_adam_pretrain = np.empty([0])
            loss_i_history_adam_pretrain = np.empty([0])
            loss_f_history_adam_pretrain = np.empty([0])

            loss_history_adam_pretrain = np.append(loss_history_adam_pretrain, loss.numpy())
            loss_b_history_adam_pretrain = np.append(loss_b_history_adam_pretrain, loss_b.numpy())
            loss_i_history_adam_pretrain = np.append(loss_i_history_adam_pretrain, loss_i.numpy())
            loss_f_history_adam_pretrain = np.append(loss_f_history_adam_pretrain, loss_f.numpy())

            start_time = time.time()
    np.save(f'Loss-Coe/train/loss_history_adam_pretrain', loss_history_adam_pretrain)
    np.save(f'Loss-Coe/train/loss_b_history_adam_pretrain', loss_b_history_adam_pretrain)
    np.save(f'Loss-Coe/train/loss_i_history_adam_pretrain', loss_i_history_adam_pretrain)
    np.save(f'Loss-Coe/train/loss_f_history_adam_pretrain', loss_f_history_adam_pretrain)

运行训练与保存训练模型

In [ ]:
for epoch, lr in zip(epochs, learning_rate):
    train(model, int(epoch), lr, batch_size=batch_size)
ms.save_checkpoint(model.dnn, f'model/{second_path}/model.ckpt')

It: 0, Total_loss: 1.583e+02, Loss_b: 6.385e-01, Loss_i: 7.558e-01, Loss_f: 1.891e+01, Time: 14.63
It: 10, Total_loss: 1.339e+02, Loss_b: 5.654e-01, Loss_i: 6.433e-01, Loss_f: 1.302e+01, Time: 4.17
It: 20, Total_loss: 8.946e+01, Loss_b: 3.713e-01, Loss_i: 4.430e-01, Loss_f: 8.031e+00, Time: 4.22
It: 30, Total_loss: 6.822e+01, Loss_b: 3.025e-01, Loss_i: 2.891e-01, Loss_f: 9.061e+00, Time: 4.23
It: 40, Total_loss: 5.122e+01, Loss_b: 2.249e-01, Loss_i: 2.195e-01, Loss_f: 6.779e+00, Time: 4.23
It: 50, Total_loss: 4.095e+01, Loss_b: 1.834e-01, Loss_i: 1.549e-01, Loss_f: 7.122e+00, Time: 4.22
It: 60, Total_loss: 3.723e+01, Loss_b: 1.664e-01, Loss_i: 1.392e-01, Loss_f: 6.662e+00, Time: 4.25
It: 70, Total_loss: 3.123e+01, Loss_b: 1.457e-01, Loss_i: 1.092e-01, Loss_f: 5.742e+00, Time: 4.26
It: 80, Total_loss: 2.830e+01, Loss_b: 1.286e-01, Loss_i: 9.858e-02, Loss_f: 5.586e+00, Time: 4.23
It: 90, Total_loss: 2.403e+01, Loss_b: 1.046e-01, Loss_i: 7.816e-02, Loss_f: 5.755e+00, Time: 4.28
It: 100, T